In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset from Kaggle
!kaggle datasets download -d salader/dogs-vs-cats

# Unzip the downloaded file
import zipfile

with zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')


Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
100% 1.06G/1.06G [00:12<00:00, 149MB/s]
100% 1.06G/1.06G [00:12<00:00, 89.4MB/s]


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os


In [3]:
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/test',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [4]:
# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

# Extract features from the images using VGG16
def extract_features(generator, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))

    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = base_model.predict(inputs_batch)
        features[i * BATCH_SIZE : (i + 1) * BATCH_SIZE] = features_batch
        labels[i * BATCH_SIZE : (i + 1) * BATCH_SIZE] = labels_batch
        i += 1
        if i * BATCH_SIZE >= sample_count:
            break

    return features, labels

train_features, train_labels = extract_features(train_generator, train_generator.samples)
train_features = np.reshape(train_features, (train_generator.samples, 4 * 4 * 512))

validation_features, validation_labels = extract_features(validation_generator, validation_generator.samples)
validation_features = np.reshape(validation_features, (validation_generator.samples, 4 * 4 * 512))


1/1 [==============================] - 2s 2s/step


In [5]:
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(train_features, train_labels)


SVC(kernel='linear')

In [6]:
validation_predictions = svm_classifier.predict(validation_features)
accuracy = accuracy_score(validation_labels, validation_predictions)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')


Validation Accuracy: 86.20%


In [7]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, IMAGE_SIZE)
    img = img.astype('float32') / 255.0
    img = np.expand_dims(img, axis=0)
    return img

test_img = preprocess_image('/content/dog1.jpg')
test_feature = base_model.predict(test_img)
test_feature = np.reshape(test_feature, (1, 4 * 4 * 512))

prediction = svm_classifier.predict(test_feature)
print('Prediction:', 'Dog' if prediction[0] == 1 else 'Cat')


1/1 [==============================] - 0s 230ms/step
Prediction: Dog
